In [ ]:
!pip install langchain replicate flask requests uvicorn gunicorn langchain_community chromadb

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 3 Community License Agreement.

!pip install flask pyngrok langchain chromadb sentence-transformers replicate
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
import requests
import json
from flask import Flask, request
from langchain.llms import Replicate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from pyngrok import ngrok

class WhatsAppClient:
    API_URL = "https://graph.facebook.com/v21.0/"
    WHATSAPP_API_TOKEN = "your_token"
    WHATSAPP_CLOUD_NUMBER_ID = "number"

    def __init__(self):
        self.headers = {
            "Authorization": f"Bearer {self.WHATSAPP_API_TOKEN}",
            "Content-Type": "application/json",
        }
        self.API_URL = self.API_URL + self.WHATSAPP_CLOUD_NUMBER_ID

    def send_text_message(self, message, phone_number):
        payload = {
            "messaging_product": 'whatsapp',
            "to": phone_number,
            "type": "text",
            "text": {
                "preview_url": False,
                "body": message
            }
        }
        response = requests.post(f"{self.API_URL}/messages", json=payload, headers=self.headers)
        print(response.status_code)
        assert response.status_code == 200, "Error sending message"
        return response.status_code

# Set up Llama 3 model
os.environ["REPLICATE_API_TOKEN"] = "your_token"
llama3_8b_chat = "meta/meta-llama-3-8b-instruct"

llm = Replicate(
    model=llama3_8b_chat,
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens": 500}
)

# Set up RAG components
embeddings = HuggingFaceEmbeddings()

# Load and process your knowledge base
# For this example, we'll create a simple text file
with open("knowledge_base.txt", "w") as f:
    f.write("This is a sample knowledge base for the RAG system.\n")
    f.write("It contains information about various topics.\n")
    f.write("The RAG system will use this information to provide context-aware responses.\n")

loader = TextLoader("knowledge_base.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Create vector store
vectorstore = Chroma.from_documents(texts, embeddings)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
)

client = WhatsAppClient()
app = Flask(__name__)

@app.route("/")
def hello_llama():
    return "<p>Hello Llama 3 RAG</p>"

@app.route('/msgrcvd', methods=['POST', 'GET'])
def msgrcvd():
    message = request.args.get('message')

    # Use the RAG system to generate a response
    result = qa_chain({"query": message})
    answer = result['result']

    print("User message:", message)
    print("Bot response:", answer)

    client.send_text_message(answer, "<your phone number>")
    return message + "<p/>" + answer

# Set up ngrok
ngrok.set_auth_token("your_token")  # Replace with your ngrok auth token
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000/\"")

if __name__ == "__main__":
    app.run()